In [1]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:2242/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

def get_llama_response(prompt, temperature = 0.1, max_tokens = 128):
    completion = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature = temperature,
        # max_tokens = max_tokens
    )

    return completion.choices[0].message.content

In [2]:
get_llama_response("৫ জন বাংলাদেশি বিখ্যাত কবির নাম বল?")

'বাংলাদেশে অনেক বিখ্যাত কবি রয়েছেন, কুড়ি ঘন্টার মধ্যে নিচের কিছু নাম প্রদান করতে পারি:\n\n1. কাজী নজরুল ইসলাম\n2. সুরেশচন্দ্র সেন\n3. জসীম উদ্দীন\n4. রবীন্দ্রনাথ ঠাকুর\n5. মঞ্জু ভূষন\n রবীন্দ্রনাথ ঠাকুর ভারতীয় কবি তবে যুগে তিনি  সারাদেশে ভাঙ্গা হেলে এসেছেন সেহেতু আজকের কাঠামোতে তিনি বাংলাদেশের কবি হিসেবেও বিবেচনা করা হয়'

In [3]:
import pandas as pd
from datasets import load_dataset

# Load the Bengali subset of the dataset
dataset = load_dataset('csebuetnlp/xlsum', 'bengali')

# Convert to pandas dataframe
df = dataset['test'].to_pandas()  # Assuming you want the training split

df = df.head(512) # we choose first 512 samples for evaluation

# Display the dataframe
df.head()

,id,url,title,summary,text
0,news-45155508,https://www.bbc.com/bengali/news-45155508,চীন ১০ লাখ উইগুর মুসলিমকে আটকে রেখেছে - জাতিসং...,জাতিসংঘের মানবাধিকার বিষয়ক কমিটি জেনেভাতে চীন...,আটক স্বজনদের আইডি কার্ড হাতে উইগুর নারীদের বিক...
1,news-39127107,https://www.bbc.com/bengali/news-39127107,"কোকা-কোলা, পেপসি নিষিদ্ধ হলো তামিলনাডুতে",ভারতের দক্ষিণাঞ্চলীয় রাজ্য তামিলনাডুর ব্যবসায...,"ভারতের অন্য অঞ্চলেও কোক, পেপসি নিষিদ্ধ করার দা..."
2,news-49141918,https://www.bbc.com/bengali/news-49141918,ডেঙ্গু পরিস্থিতি: হাসপাতালে ভর্তির দরকার না থা...,"স্বাস্থ্য অধিদপ্তরের হিসাব মতে, গত ২৪ ঘণ্টায় ...",সোহরাওয়ার্দী হাসপাতালের চিত্র সাধারণত সেপ্টেম...
3,news-52808250,https://www.bbc.com/bengali/news-52808250,ডোনাল্ড ট্রাম্প: আমেরিকান প্রেসিডেন্টের পোস্ট ...,এই প্রথম সামাজিক মাধ্যমের বিশাল প্রতিষ্ঠান টুই...,ডোনাল্ড ট্রাম্প অভিযোগ করেছেন টু্‌ইটার আমেরিকা...
4,news-49047323,https://www.bbc.com/bengali/news-49047323,বাংলাদেশে বন্যার প্রকোপ: ২৪ ঘণ্টায় ডুবে যাবে ...,বাংলাদেশের বন্যা পূর্বাভাস এবং সতর্কীকরণ কেন্দ...,বন্যার প্রকোপ ছড়িয়ে পড়ছে সারা দেশে। বাংলাদে...


In [4]:
zero_shot = """আপনাকে একটি দীর্ঘ বাংলা বিবরণ দেওয়া হয়েছে। আপনাকে এই বিবরণের ১ বা ২ বাক্যে সারমর্ম লিখতে হবে।

"""

In [5]:
three_shot = """আপনাকে একটি দীর্ঘ বাংলা বিবরণ দেওয়া হয়েছে। আপনাকে এই বিবরণের ১ বা ২ বাক্যে সারমর্ম লিখতে হবে।

### উদাহরণ ১
### বিবরণ:
আলজেরিয়ায় পরীক্ষার হল স্কুলের ছাত্রছাত্রীরা যাতে পরীক্ষার হলে নকল করতে না পারে কিম্বা আগেভাগেই প্রশ্ন না পেতে না পারে সেজন্যে কর্তৃপক্ষ সাময়িকভাবে দেশটিতে সোশাল মিডিয়া বন্ধ করে দিয়েছে। এর আগে স্কুলের পরীক্ষায় অনলাইনের মাধ্যমে বহু প্রশ্ন ফাঁস হয়ে গেলে দেশটিতে কর্তৃপক্ষের তীব্র সমালোচনা হয়। তখন অর্ধেকেরও বেশি শিক্ষার্থীকে দ্বিতীয়বার পরীক্ষায় বসত হয়েছে। সংবাদ মাধ্যমগুলোর খবরে বলা হচ্ছে, জুন মাসে বহু ছাত্রছাত্রীই পরীক্ষার আগে ফেসবুকে ও অন্যান্য সোশাল মিডিয়া থেকে প্রশ্ন হাতে পেয়ে যায়। এর আগের বছরগুলোতেও দেশটিতে এধরনের ঘটনা ঘটেছে। সরকারি কর্মকর্তারা বলছেন, পরীক্ষার্থীদেরকে নকল ও প্রশ্ন ফাঁসের হাত থেকে রক্ষার জন্যেই সোশাল মিডিয়া ব্লক করে দেওয়ার সিদ্ধান্ত নেওয়া হয়েছে।

### সারমর্ম:
আফ্রিকার দেশ আলজেরিয়ায় পরীক্ষায় নকল ও প্রশ্ন ফাঁস করতে নজিরবিহীন এক উদ্যোগ নিয়েছে।

### উদাহরণ ২
### বিবরণ:
চ্যাম্পিয়ন্স লিগে এক ক্লাবের হয়ে ১০০ গোল করা একমাত্র খেলোয়াড় রোনালদো বুধবার চ্যাম্পিয়ন্স লিগে পিএসজির বিপক্ষে ৩-১এর জয়ে ২ গোল করে এক ক্লাবের হয়ে ১০০ চ্যাম্পিয়ন্স লিগ গোল করা একমাত্র খেলোয়াড় হয়েছেন রেয়াল মাদ্রিদের ক্রিশ্চিয়ানো রোনালদো। এই তালিকায় দ্বিতীয় স্থানে রয়েছেন তাঁর চেয়ে ৪ গোল কম করা লিওনেল মেসি। রিয়াল মাদ্রিদের হয়ে ১০০ গোলের পর রোনালদো চ্যাম্পিয়ন্স লিগে রোনালদোর আরো রেকর্ড রেয়াল মাদ্রিদে যোগ দেয়ার আগে ম্যানচেস্টার ইউনাইটেডের হয়ে করা ১৫টি গোল মিলিয়ে চ্যাম্পিয়ন্স লিগে রোনালদোর মোট গোল ১১৬টি। যা দ্বিতীয় স্থানে থাকা মেসির চেয়ে ১৯টি বেশী। চ্যাম্পিয়ন্স লিগের ইতিহাসে এই টুর্নামেন্টে খেলা ১৩৭টি ক্লাবের ১১৮টির মোট গোলসংখ্যা রোনালদোর একার গোলের চেয়ে কম।

### সারমর্ম:
১০০ গোলের ক্লাবে একমাত্র সদস্য - ক্রিশ্চিয়ানো রোনালদো।

### উদাহরণ ৩
### বিবরণ:
বৈরুতের বন্দরের নিকটবর্তী অঞ্চল সবচেয়ে বেশি ক্ষতিগ্রস্ত হয় বিস্ফোরণে এমনও খবর পাওয়া যাচ্ছে যে কয়েক'শো কিলোমিটার দূরের সাইপ্রাসেও ঐ বিস্ফোরণের আঁচ পাওয়া গেছে। কর্মকর্তারা বলছেন এই বিস্ফোরণ দুর্ঘটনা। পরিকল্পিতভাবে এই বিস্ফোরণ ঘটানো হয়নি। তারা বলছেন গুদামে ছয় বছর ধরে মজুত রাখা অত্যন্ত বিপদজনক বিস্ফোরক থেকে এই দুর্ঘটনা ঘটেছে। তবে ঠিক কী কারণে এই বিস্ফোরণ ঘটেছে, তা পরিস্কার নয়। ক্ষতিগ্রস্থ ভবনের সামনে লেবাননের 'লেবানিজ এমিগ্র্যান্ট' স্ট্যাচু বহু ঘরবাড়ি ধ্বংস হয়ে গেছে বিস্ফোরণে বিস্ফোরণে পর বৈরুতের আকাশে ধোঁয়া আহত এক ব্যক্তিকে সরিয়ে নিয়ে যাচ্ছেন উদ্ধারকারীরা বিস্ফোরণে আহত হয়েছেন হাজার হাজার মানুষ স্থানীয় সংবাদমাধ্যমের খবরে ধ্বংসস্তুপের নিচে মানুষকে আটকা পড়ে থাকতে দেখা গেছে। বিস্ফোরণের পর রাস্তাগুলোও যেন ধ্বংসস্তূপে পরিণত হয় বিস্ফোরণের স্থান থেকে নির্গত হচ্ছে ধোঁয়া সব ছবির কপিরাইট রয়েছে

### সারমর্ম:
বিশাল এক বিস্ফোরণ ব্যাপক ধ্বংসযজ্ঞ চালিয়েছে লেবাননের রাজধানী বৈরুতে। বহু মানুষ মারা গেছে, আহত হয়েছে কয়েক হাজারের বেশি।

### আপনার কাজ

"""

In [6]:
chain_of_thought = """আপনাকে একটি দীর্ঘ বাংলা বিবরণ দেওয়া হয়েছে। আপনাকে এই বিবরণের ১ বা ২ বাক্যে সারমর্ম লিখতে হবে। সারমর্ম লিখতে গিয়ে নিম্নলিখিত ধাপগুলি অনুসরণ করুন:

### ধাপ ১: মূল বিষয়বস্তু বোঝা
বিবরণটি পড়ুন এবং এর মূল বিষয় বা ঘটনাটি বোঝার চেষ্টা করুন। প্রধান তথ্য বা ঘটনা শনাক্ত করুন যা বিবরণের সারমর্ম হতে পারে।

### ধাপ ২: প্রধান তথ্য চিহ্নিত করা
বিবরণের প্রধান তথ্য বা ঘটনার সংক্ষিপ্ত রূপ বের করুন। কোন বিষয়গুলি সবচেয়ে গুরুত্বপূর্ণ তা চিহ্নিত করুন।

### ধাপ ৩: সারমর্ম লেখা
বিবরণের প্রধান তথ্যগুলি ব্যবহার করে ১ বা ২ বাক্যে সারমর্ম লিখুন। এটি সংক্ষিপ্ত এবং সুনির্দিষ্ট হতে হবে।

### উদাহরণ ১
### বিবরণ:
আলজেরিয়ায় পরীক্ষার হল স্কুলের ছাত্রছাত্রীরা যাতে পরীক্ষার হলে নকল করতে না পারে কিম্বা আগেভাগেই প্রশ্ন না পেতে না পারে সেজন্যে কর্তৃপক্ষ সাময়িকভাবে দেশটিতে সোশাল মিডিয়া বন্ধ করে দিয়েছে। এর আগে স্কুলের পরীক্ষায় অনলাইনের মাধ্যমে বহু প্রশ্ন ফাঁস হয়ে গেলে দেশটিতে কর্তৃপক্ষের তীব্র সমালোচনা হয়। তখন অর্ধেকেরও বেশি শিক্ষার্থীকে দ্বিতীয়বার পরীক্ষায় বসত হয়েছে। সংবাদ মাধ্যমগুলোর খবরে বলা হচ্ছে, জুন মাসে বহু ছাত্রছাত্রীই পরীক্ষার আগে ফেসবুকে ও অন্যান্য সোশাল মিডিয়া থেকে প্রশ্ন হাতে পেয়ে যায়। এর আগের বছরগুলোতেও দেশটিতে এধরনের ঘটনা ঘটেছে। সরকারি কর্মকর্তারা বলছেন, পরীক্ষার্থীদেরকে নকল ও প্রশ্ন ফাঁসের হাত থেকে রক্ষার জন্যেই সোশাল মিডিয়া ব্লক করে দেওয়ার সিদ্ধান্ত নেওয়া হয়েছে।

#### চিন্তাধারা:
- মূল বিষয়বস্তু: পরীক্ষার হলে নকল ও প্রশ্ন ফাঁস ঠেকাতে সোশাল মিডিয়া বন্ধ।
- প্রধান তথ্য: নকল ও প্রশ্ন ফাঁস ঠেকাতে আলজেরিয়ায় সোশাল মিডিয়া সাময়িকভাবে বন্ধ।

### সারমর্ম:
আফ্রিকার দেশ আলজেরিয়ায় পরীক্ষায় নকল ও প্রশ্ন ফাঁস করতে নজিরবিহীন এক উদ্যোগ নিয়েছে।

### উদাহরণ ২
### বিবরণ:
চ্যাম্পিয়ন্স লিগে এক ক্লাবের হয়ে ১০০ গোল করা একমাত্র খেলোয়াড় রোনালদো বুধবার চ্যাম্পিয়ন্স লিগে পিএসজির বিপক্ষে ৩-১এর জয়ে ২ গোল করে এক ক্লাবের হয়ে ১০০ চ্যাম্পিয়ন্স লিগ গোল করা একমাত্র খেলোয়াড় হয়েছেন রেয়াল মাদ্রিদের ক্রিশ্চিয়ানো রোনালদো। এই তালিকায় দ্বিতীয় স্থানে রয়েছেন তাঁর চেয়ে ৪ গোল কম করা লিওনেল মেসি। রিয়াল মাদ্রিদের হয়ে ১০০ গোলের পর রোনালদো চ্যাম্পিয়ন্স লিগে রোনালদোর আরো রেকর্ড রেয়াল মাদ্রিদে যোগ দেয়ার আগে ম্যানচেস্টার ইউনাইটেডের হয়ে করা ১৫টি গোল মিলিয়ে চ্যাম্পিয়ন্স লিগে রোনালদোর মোট গোল ১১৬টি। যা দ্বিতীয় স্থানে থাকা মেসির চেয়ে ১৯টি বেশী। চ্যাম্পিয়ন্স লিগের ইতিহাসে এই টুর্নামেন্টে খেলা ১৩৭টি ক্লাবের ১১৮টির মোট গোলসংখ্যা রোনালদোর একার গোলের চেয়ে কম।

#### চিন্তাধারা:
- মূল বিষয়বস্তু: রোনালদোর ১০০ গোল করা।
- প্রধান তথ্য: এক ক্লাবের হয়ে ১০০ গোল করা একমাত্র খেলোয়াড় রোনালদো।

### সারমর্ম:
১০০ গোলের ক্লাবে একমাত্র সদস্য - ক্রিশ্চিয়ানো রোনালদো।

### উদাহরণ ৩
### বিবরণ:
বৈরুতের বন্দরের নিকটবর্তী অঞ্চল সবচেয়ে বেশি ক্ষতিগ্রস্ত হয় বিস্ফোরণে এমনও খবর পাওয়া যাচ্ছে যে কয়েক'শো কিলোমিটার দূরের সাইপ্রাসেও ঐ বিস্ফোরণের আঁচ পাওয়া গেছে। কর্মকর্তারা বলছেন এই বিস্ফোরণ দুর্ঘটনা। পরিকল্পিতভাবে এই বিস্ফোরণ ঘটানো হয়নি। তারা বলছেন গুদামে ছয় বছর ধরে মজুত রাখা অত্যন্ত বিপদজনক বিস্ফোরক থেকে এই দুর্ঘটনা ঘটেছে। তবে ঠিক কী কারণে এই বিস্ফোরণ ঘটেছে, তা পরিস্কার নয়। ক্ষতিগ্রস্থ ভবনের সামনে লেবাননের 'লেবানিজ এমিগ্র্যান্ট' স্ট্যাচু বহু ঘরবাড়ি ধ্বংস হয়ে গেছে বিস্ফোরণে বিস্ফোরণে পর বৈরুতের আকাশে ধোঁয়া আহত এক ব্যক্তিকে সরিয়ে নিয়ে যাচ্ছেন উদ্ধারকারীরা বিস্ফোরণে আহত হয়েছেন হাজার হাজার মানুষ স্থানীয় সংবাদমাধ্যমের খবরে ধ্বংসস্তুপের নিচে মানুষকে আটকা পড়ে থাকতে দেখা গেছে। বিস্ফোরণের পর রাস্তাগুলোও যেন ধ্বংসস্তূপে পরিণত হয় বিস্ফোরণের স্থান থেকে নির্গত হচ্ছে ধোঁয়া সব ছবির কপিরাইট রয়েছে

#### চিন্তাধারা:
- মূল বিষয়বস্তু: বৈরুতের বিস্ফোরণ।
- প্রধান তথ্য: বিস্ফোরণে বৈরুতে ব্যাপক ধ্বংসযজ্ঞ ও বহু মানুষ আহত।

### সারমর্ম:
বিশাল এক বিস্ফোরণ ব্যাপক ধ্বংসযজ্ঞ চালিয়েছে লেবাননের রাজধানী বৈরুতে। বহু মানুষ মারা গেছে, আহত হয়েছে কয়েক হাজারের বেশি।

### আপনার কাজ

"""

In [7]:
path_root = 'exps/'

In [8]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_summary'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet(path_root + 'llama_zs_summary_results.parquet')
    start_index = df_progress['predicted_summary'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    text_body = row['text']

    prompt_input = zero_shot + " বিবরণ: " + text_body + "\nসারমর্ম:"

    try:
        df.at[i, 'predicted_summary'] = get_llama_response(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet(path_root + 'llama_zs_summary_results.parquet')

# Save final progress
df.to_parquet(path_root + 'llama_zs_summary_results.parquet')

  0%|          | 0/512 [00:00<?, ?it/s]

In [9]:
zs_results = pd.read_parquet(path_root + 'llama_zs_summary_results.parquet')
zs_results.head()

,id,url,title,summary,text,predicted_summary
0,news-45155508,https://www.bbc.com/bengali/news-45155508,চীন ১০ লাখ উইগুর মুসলিমকে আটকে রেখেছে - জাতিসং...,জাতিসংঘের মানবাধিকার বিষয়ক কমিটি জেনেভাতে চীন...,আটক স্বজনদের আইডি কার্ড হাতে উইগুর নারীদের বিক...,চীনে উইগুর এবং অন্যান্য মুসলিমদের বিরুদ্ধে চীন...
1,news-39127107,https://www.bbc.com/bengali/news-39127107,"কোকা-কোলা, পেপসি নিষিদ্ধ হলো তামিলনাডুতে",ভারতের দক্ষিণাঞ্চলীয় রাজ্য তামিলনাডুর ব্যবসায...,"ভারতের অন্য অঞ্চলেও কোক, পেপসি নিষিদ্ধ করার দা...",তামিলনাডু রাজ্যে কোকা-কোলা এবং পেপসি নিষিদ্ধ ক...
2,news-49141918,https://www.bbc.com/bengali/news-49141918,ডেঙ্গু পরিস্থিতি: হাসপাতালে ভর্তির দরকার না থা...,"স্বাস্থ্য অধিদপ্তরের হিসাব মতে, গত ২৪ ঘণ্টায় ...",সোহরাওয়ার্দী হাসপাতালের চিত্র সাধারণত সেপ্টেম...,ডেঙ্গু আক্রান্ত রোগীর সংখ্যা দিন দিন বাড়ছে এব...
3,news-52808250,https://www.bbc.com/bengali/news-52808250,ডোনাল্ড ট্রাম্প: আমেরিকান প্রেসিডেন্টের পোস্ট ...,এই প্রথম সামাজিক মাধ্যমের বিশাল প্রতিষ্ঠান টুই...,ডোনাল্ড ট্রাম্প অভিযোগ করেছেন টু্‌ইটার আমেরিকা...,"বিবিসির খবর অনুযায়ী, বিশ্বের প্রসিডেন্ট ডোনাল..."
4,news-49047323,https://www.bbc.com/bengali/news-49047323,বাংলাদেশে বন্যার প্রকোপ: ২৪ ঘণ্টায় ডুবে যাবে ...,বাংলাদেশের বন্যা পূর্বাভাস এবং সতর্কীকরণ কেন্দ...,বন্যার প্রকোপ ছড়িয়ে পড়ছে সারা দেশে। বাংলাদে...,বাংলাদেশের সারা দেশে বন্যার প্রকোপ ছড়িয়ে পড়...


In [10]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_summary'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet(path_root + 'llama_ts_summary_results.parquet')
    start_index = df_progress['predicted_summary'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    text_body = row['text']

    prompt_input = three_shot + "### বিবরণ: " + text_body + "\n### সারমর্ম:"

    try:
        df.at[i, 'predicted_summary'] = get_llama_response(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet(path_root + 'llama_ts_summary_results.parquet')

# Save final progress
df.to_parquet(path_root + 'llama_ts_summary_results.parquet')

  0%|          | 0/512 [00:00<?, ?it/s]

In [11]:
ts_results = pd.read_parquet(path_root + 'llama_ts_summary_results.parquet')
ts_results.head()

,id,url,title,summary,text,predicted_summary
0,news-45155508,https://www.bbc.com/bengali/news-45155508,চীন ১০ লাখ উইগুর মুসলিমকে আটকে রেখেছে - জাতিসং...,জাতিসংঘের মানবাধিকার বিষয়ক কমিটি জেনেভাতে চীন...,আটক স্বজনদের আইডি কার্ড হাতে উইগুর নারীদের বিক...,উইগুর স্বজনদের আইডি কার্ড হাতে উইগুর নারীদের ব...
1,news-39127107,https://www.bbc.com/bengali/news-39127107,"কোকা-কোলা, পেপসি নিষিদ্ধ হলো তামিলনাডুতে",ভারতের দক্ষিণাঞ্চলীয় রাজ্য তামিলনাডুর ব্যবসায...,"ভারতের অন্য অঞ্চলেও কোক, পেপসি নিষিদ্ধ করার দা...",তামিলনাড়ুর ব্যবসায়ীরা দেশের উপর থেকে আয়োজিত...
2,news-49141918,https://www.bbc.com/bengali/news-49141918,ডেঙ্গু পরিস্থিতি: হাসপাতালে ভর্তির দরকার না থা...,"স্বাস্থ্য অধিদপ্তরের হিসাব মতে, গত ২৪ ঘণ্টায় ...",সোহরাওয়ার্দী হাসপাতালের চিত্র সাধারণত সেপ্টেম...,বাংলাদেশের উষ্ণ ও আর্দ্র আবহাবায় ডেঙ্গু রোগের...
3,news-52808250,https://www.bbc.com/bengali/news-52808250,ডোনাল্ড ট্রাম্প: আমেরিকান প্রেসিডেন্টের পোস্ট ...,এই প্রথম সামাজিক মাধ্যমের বিশাল প্রতিষ্ঠান টুই...,ডোনাল্ড ট্রাম্প অভিযোগ করেছেন টু্‌ইটার আমেরিকা...,যুক্তরাষ্ট্রের ডোনাল্ড ট্রাম্প প্রেসিডেন্টের ব...
4,news-49047323,https://www.bbc.com/bengali/news-49047323,বাংলাদেশে বন্যার প্রকোপ: ২৪ ঘণ্টায় ডুবে যাবে ...,বাংলাদেশের বন্যা পূর্বাভাস এবং সতর্কীকরণ কেন্দ...,বন্যার প্রকোপ ছড়িয়ে পড়ছে সারা দেশে। বাংলাদে...,"বাংলাদেশে ২২টি জেলা বন্যাপ্লাবিত হয়েছে, যার ম..."


In [ ]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
if 'predicted_category' not in df:
    df['predicted_summary'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet(path_root + 'llama_cot_summary_results.parquet')
    start_index = df_progress['predicted_summary'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0

start_index = 0

for i, row in tqdm(df_progress.iterrows()):
    text_body = row['text']

    prompt_input = chain_of_thought + "### বিবরণ: " + text_body

    if df_progress.at[i, 'predicted_summary'] != None:
        continue
        
    try:
        df_progress.at[i, 'predicted_summary'] = get_llama_response(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df_progress.to_parquet(path_root + 'llama_cot_summary_results.parquet')

# Save final progress
df_progress.to_parquet(path_root + 'llama_cot_summary_results.parquet')

0it [00:00, ?it/s]

In [8]:
df_progress = pd.read_parquet(path_root + 'llama_cot_summary_results.parquet')

In [11]:
df_progress['predicted_summary'].last_valid_index() + 1

200

In [1]:
! pip install rouge bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 48.9 MB/s eta 0:00:00


In [2]:
! pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 46.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
from rouge import Rouge
from bert_score import score as bert_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def evaluate_summarization(df):
    # Replace None values in 'predicted_summary' with empty strings
    df['predicted_summary'] = df['predicted_summary'].fillna("")

    # Filter out empty summaries
    non_empty_df = df[df['summary'].str.strip() != ""]
    non_empty_df = non_empty_df[non_empty_df['predicted_summary'].str.strip() != ""]

    # Initialize ROUGE
    rouge = Rouge()

    # Calculate ROUGE-L score
    rouge_l_scores = rouge.get_scores(non_empty_df['predicted_summary'].tolist(), non_empty_df['summary'].tolist(), avg=True)['rouge-l']
    avg_rouge_l_f = rouge_l_scores['f']
    avg_rouge_l_precision = rouge_l_scores['p']
    avg_rouge_l_recall = rouge_l_scores['r']

    # Calculate BERTScore
    non_empty_refs = non_empty_df['summary'].tolist()
    non_empty_preds = non_empty_df['predicted_summary'].tolist()

    if non_empty_refs and non_empty_preds:
        P, R, F1 = bert_score(non_empty_preds, non_empty_refs, lang='bn', device='cpu')
        bert_scores = {
            'bert_precision': P.mean().item(),
            'bert_recall': R.mean().item(),
            'bert_f1': F1.mean().item()
        }
    else:
        bert_scores = {
            'bert_precision': 0,
            'bert_recall': 0,
            'bert_f1': 0
        }

    # Calculate BLEU score
    smoothie = SmoothingFunction().method4
    bleu_scores = [sentence_bleu([ref], pred, smoothing_function=smoothie) for ref, pred in zip(non_empty_refs, non_empty_preds)]
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0

    metrics = {
        'rouge_l_f': avg_rouge_l_f,
        'rouge_l_precision': avg_rouge_l_precision,
        'rouge_l_recall': avg_rouge_l_recall,
        'bert_precision': bert_scores['bert_precision'],
        'bert_recall': bert_scores['bert_recall'],
        'bert_f1': bert_scores['bert_f1'],
        'bleu': avg_bleu_score
    }

    return metrics

In [5]:
path_root = 'exps/'

In [6]:
zs_df = pd.read_parquet(path_root + "llama_zs_summary_results.parquet")
# zs_df['predicted_summary'] = zs_df['predicted_summary'].fillna("")
metrics = evaluate_summarization(zs_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/opt/conda/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

rouge_l_f: 0.1150
rouge_l_precision: 0.1007
rouge_l_recall: 0.1542
bert_precision: 0.6954
bert_recall: 0.7228
bert_f1: 0.7083
bleu: 0.2663


In [7]:
ts_df = pd.read_parquet(path_root + "llama_ts_summary_results.parquet")
# zs_df['predicted_summary'] = zs_df['predicted_summary'].fillna("")
metrics = evaluate_summarization(ts_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

/opt/conda/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


rouge_l_f: 0.1085
rouge_l_precision: 0.1113
rouge_l_recall: 0.1232
bert_precision: 0.7031
bert_recall: 0.7100
bert_f1: 0.7059
bleu: 0.2570


In [8]:
cot_df = pd.read_parquet(path_root + "llama_cot_summary_results.parquet")
# zs_df['predicted_summary'] = zs_df['predicted_summary'].fillna("")
metrics = evaluate_summarization(cot_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

/opt/conda/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


rouge_l_f: 0.0783
rouge_l_precision: 0.0655
rouge_l_recall: 0.1535
bert_precision: 0.6416
bert_recall: 0.6945
bert_f1: 0.6659
bleu: 0.1660
